In [21]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import boto3

from IPython.display import HTML, display

ModuleNotFoundError: No module named 'boto3'

In [ ]:
#quick warmup

# imagine a key-value data container
inventory = {
      "001": [{"name": "Milk", "quantity": 34, "price": 1.99}],
      "002": [{"name": "Bread", "quantity": 20, "price": 2.49},
              {"name": "Nutella", "quantity": 5, "price": 2.49}]    
    }

inventory

In [ ]:
#How would a store manager find out how much is milk

#How do we add new item? What do we need to do?

In [ ]:
name_to_item = {}
for item_id, details_list in inventory.items():
    for details in details_list:
        # Here we are assuming product names are unique
        name_to_item[details["name"]] = {"id": item_id, "quantity": details["quantity"], "price": details["price"]}
#if details["name"] is not unique, what happens?
print(name_to_item)

# Lecture 05 - JSON, XML, HTML, Requests and APIs
by Jan Šíla, Vítek Macháček <br />
March 26, 2024

### Contents

* Standardized data representation
* JSON
* XML
* Introduction to BeautifulSoup
* Basics of HTML (+ Element Inspection)
* Introduction to Requests (GET vs. POST) and APIs


### Goals:
    
* work with data  online/real-time data
* acquisition, processing - > results

## Date exchange formats - JSON, XML

`Language of the internet`

* You can send/receive a message with (almost) any service

* send .docx -> what if I do not have MS Word?
* we need a simple data format which would work on any machine (system agnostic), is general (can write anything) and is ediatable in basic editors

* More complex than simple tables
* Highly structured - if you dont follow the rules, you are out
* Both sides need to understand the structure (comments in yaml)
* only data, no code to be run (security measure)
* distributed as text/string (to be precise as `bytes` literals) 
* parsed to objects - easy to work with straight away
* Can be persisted as special files, or some data streams from APIs. 
* Human readable
* Hierarchical
* Can be fetched using standard web APIs

### Purpose

1. Communication 
    * All imaginable communication channels
    * Applications within single server/machine
    * Only transferring of data
    * Both sides need to understand the structure

2. Storing
    * self-descriptive
    * human readable
    * also in DBs - SQL, MongoDB etc.

3. Standardization
    * predictability
    * cooperation
    * spillovers from standardization


### Dimensionality problem

* rich information comes at costs of data complexity 
* to interrelate information, you need to high dimensionality (or A LOT of columns) or declaratory formats such as protobuf
* Strongly object-oriented


### 1D:
* logs

### 2D: CSVs
* tabular data (like pandas DFs)

### 3+D:
#### XML
* eXtensible Markup Language is a software- and hardware-independent tool for storing and transporting data.
* Officialy defined at 1998, but its roots are even older.
* XML was designed to carry data - with focus on what data is
* HTML was designed to display data - with focus on what data should look like displayed 
* XML tags are not predefined like HTML tags are
* more verbose than JSON
* can have comments !actually a really cool in useful feature!
* used historically as a transaction format in many areas: 
    * Scientific measurements
    * News information
    * Wheather measurements
    * Financial transactions
* Necessary to use XML parser to use in Python or in JavaScript


### JSON
* JavaScript Object Notation
* often *.json* files
* but also used in the web etc.
* supports standard datatypes - strings, integers, floats, lists
* No comments
* More compact, less verbose
* No closing tags
* Used EVERYWHERE, BUT [NOT LICENSED FOR EVIL](https://www.json.org/license.html). If you want to do evil stuff, use XML instead.
* Native in JavaScript and close to native in Python (dictionary)
* Jupyter Notebooks


* common pitfals: properly formatted JSON is different to python dict. -> check: https://jsonlint.com/


# JSON

In [ ]:
# general representation of a dictionary
# emphasis on accessibility -> key-value ( hash table )
# contains records, lists, or other dictionaries

teachers = [
    {'name':'Jozef Baruník','titles':['doc.','PhDr.','Ph.D.','Bc.','Mgr.'],'ID':1234,'courses':['JEM005','JEM116','JEM059','JEM061']},
    {'name':'Martin Hronec','titles':['Bc.','Mgr.'],'ID':3421,'courses':['JEM005','JEM207']},
]

courses = {
    "JEM005":{'name':'Advanced Econometrics','ECTS':6,'teachers':[3421,1234]},
    'JEM207':{'name':'Data Processing in Python','ECTS':5,'teachers':[3421]},
    'JEM116':{'name':'Applied Econometrics','ECTS':6,'teachers':[1234]},
    'JEM059':{'name':'Quantitative Finance I.','ECTS':6,'teachers':[1234,5678]},
    'JEM061':{'name':'Quantitative Finance II.','ECTS':6,'teachers':[1234,5678]}
}
jsondata = {'teachers':teachers,'courses':courses}
jsondata

is this a valid JSON?

https://jsonformatter.curiousconcept.com/

![python and JSON](./06_pics/python_json.png)

In [ ]:
js = json.dumps(
    jsondata
) #json formatted string!

isinstance(js,str)


In [ ]:
js

In [ ]:
jsondata['courses']['JEM005']['test']='test'

In [ ]:
pd.DataFrame(jsondata['courses']).transpose()

In [ ]:
dfc = pd.read_json(json.dumps(jsondata['courses']),orient='index')
dfc

In [ ]:
# lets come back to this a little later

# eXtensible Markup Language (XML)

* elements
* attributes
* tags

### Tag
> <>

### Element

### Convert to python data-types

In [ ]:
#either
'''<element>content</element>'''

#or self-closing (no content)
'''<element />''';
# <br />

### Attributes

In [ ]:
'''<element attr="value" />''';

![XML tree structure](./06_pics/xml_tree_structure.png)

```xml
<bookstore>
    <book category="fiction">
        <title lang="ENG">Everyday Italian</title>
        <title lang="CZE">AAaAA</title>
        <author>Giada De Laurentis</author>
        <year>2005</year>
        <price>30.00</price>
    </book>
</bookstore>
```

```json
{
    "bookstore":[
        {
            "title":"Everyday Italian",
            "lang":"ENG",
            "author":"Giada de Laurentis",
            "year":2005,
            "price":30
        }
    ]
}
```


Takeaway: JSON and XML are not equivalents and cannot be freely mirrored. Unfortunately.

JSON cannot have multiple tags with different properties ->title_en, title_cze  perhaps

## Navigation
* Xpath
* CSS selectors 
* **BeautifulSoup**

### BeatifulSoup in detail
each BS object represents
* an element
* the position in tree

In [ ]:
'''string  on more 
nes '''

In [ ]:
xml = '''
<?xml version="1.0" encoding="utf-8"?>
<ies_data>
    <courses>
        <course id="JEM005" ects="6" name="Advanced Econometrics">
           <teacher-id>3421</teacher-id>
           <teacher-id>1234</teacher-id>
        </course>
        <course id="JEM207" ects="5" name="Data Processing in Python">
            <teacher-id>3421</teacher-id>
        </course>
            <course id="JEM116" ects="6" name="Applied Econometrics I.">
            <teacher-id>1234</teacher-id>
        </course>
        <course id="JEM059" ects="6" name="Quantitative Finance I.">
            <teacher-id>1234</teacher-id>
            <teacher-id>5678</teacher-id>
        </course>
        <course id="JEM061" ects="6" name="Quantitative Finance II.">
            <teacher-id>1234</teacher-id>
            <teacher-id>5678</teacher-id>
        </course>
    </courses>
    <teachers>
        <teacher teacher-id="3421">
            <name>Martin Hronec</name>
        </teacher>
        <teacher teacher-id="1234">
            <name>Jozef Baruník</name>
        </teacher>
        <teacher teacher-id="5678">
            <name>Lukáš Vácha</name>
        </teacher>
    </teachers>
</ies_data>
'''

#unlike HTML, those tag names are defined by Vitek - no one else 'can' understand them -> flexibility is limited. But same issue with JSON to be fair

soup = BeautifulSoup(xml)

In [ ]:
dir(soup)

```find()``` will find a **first** element given the input

```find_all()``` or ```findAll()```  finds a **all** elements given the input

In [22]:
soup.find_all('course')[0].find('teacher-id')

<teacher-id>3421</teacher-id>

In [24]:
jem059 = soup.find('course',{'id':'JEM059'}) #looking for a tag with attrbitues (optional)

In [25]:
jem059

<course ects="6" id="JEM059" name="Quantitative Finance I.">
<teacher-id>1234</teacher-id>
<teacher-id>5678</teacher-id>
</course>

In [26]:
soup.findAll('teacher-id')

[<teacher-id>3421</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>3421</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>5678</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>5678</teacher-id>]

`soup['attr']` will return the value of attribute

In [27]:
print(jem059['ects'])
print(jem059['name'])

6
Quantitative Finance I.


In [28]:
soup.findAll('teacher-id')

[<teacher-id>3421</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>3421</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>5678</teacher-id>,
 <teacher-id>1234</teacher-id>,
 <teacher-id>5678</teacher-id>]

In [29]:
jem059

<course ects="6" id="JEM059" name="Quantitative Finance I.">
<teacher-id>1234</teacher-id>
<teacher-id>5678</teacher-id>
</course>

you can also navigate horizontally

In [30]:
jem059.findNext('course').findNext('course')

In [34]:
jem059.findPrevious('course').findPrevious('course')

<course ects="5" id="JEM207" name="Data Processing in Python">
<teacher-id>3421</teacher-id>
</course>

and even upstream!

In [36]:
jem059.parent.parent

<ies_data>
<courses>
<course ects="6" id="JEM005" name="Advanced Econometrics">
<teacher-id>3421</teacher-id>
<teacher-id>1234</teacher-id>
</course>
<course ects="5" id="JEM207" name="Data Processing in Python">
<teacher-id>3421</teacher-id>
</course>
<course ects="6" id="JEM116" name="Applied Econometrics I.">
<teacher-id>1234</teacher-id>
</course>
<course ects="6" id="JEM059" name="Quantitative Finance I.">
<teacher-id>1234</teacher-id>
<teacher-id>5678</teacher-id>
</course>
<course ects="6" id="JEM061" name="Quantitative Finance II.">
<teacher-id>1234</teacher-id>
<teacher-id>5678</teacher-id>
</course>
</courses>
<teachers>
<teacher teacher-id="3421">
<name>Martin Hronec</name>
</teacher>
<teacher teacher-id="1234">
<name>Jozef Baruník</name>
</teacher>
<teacher teacher-id="5678">
<name>Lukáš Vácha</name>
</teacher>
</teachers>
</ies_data>

In [38]:
#get all teacher ids
teacher_ids = [int(t.text) for t in soup.findAll('teacher-id')]
print(teacher_ids)
#get unique
set(teacher_ids)

[3421, 1234, 3421, 1234, 1234, 5678, 1234, 5678]


{1234, 3421, 5678}

In [39]:
course = soup.find('course')
d = {
    'id':course['id'],
    'name':course['name'],
    'ects':course['ects'],
    'teachers':[int(t.text) for t in course.findAll('teacher-id')]
}
d

{'id': 'JEM005',
 'name': 'Advanced Econometrics',
 'ects': '6',
 'teachers': [3421, 1234]}

### Can convert to JSON-like

In [40]:
l = []
for course in soup.findAll('course'):
    d = {'id':course['id'],
         'name':course['name'],
         'ects':course['ects'],
         'teachers':[int(t.text) for t in course.findAll('teacher-id')]}
    l.append(d)
l

[{'id': 'JEM005',
  'name': 'Advanced Econometrics',
  'ects': '6',
  'teachers': [3421, 1234]},
 {'id': 'JEM207',
  'name': 'Data Processing in Python',
  'ects': '5',
  'teachers': [3421]},
 {'id': 'JEM116',
  'name': 'Applied Econometrics I.',
  'ects': '6',
  'teachers': [1234]},
 {'id': 'JEM059',
  'name': 'Quantitative Finance I.',
  'ects': '6',
  'teachers': [1234, 5678]},
 {'id': 'JEM061',
  'name': 'Quantitative Finance II.',
  'ects': '6',
  'teachers': [1234, 5678]}]

### Or in list-comprehension syntax

In [42]:
l = [{
    'id':course['id'],
    'name':course['name'],
    'ects':course['ects'],
    'teachers':[int(t.text) for t in course.findAll('teacher-id')]
} for course in soup.findAll('course')]

In [43]:
l

[{'id': 'JEM005',
  'name': 'Advanced Econometrics',
  'ects': '6',
  'teachers': [3421, 1234]},
 {'id': 'JEM207',
  'name': 'Data Processing in Python',
  'ects': '5',
  'teachers': [3421]},
 {'id': 'JEM116',
  'name': 'Applied Econometrics I.',
  'ects': '6',
  'teachers': [1234]},
 {'id': 'JEM059',
  'name': 'Quantitative Finance I.',
  'ects': '6',
  'teachers': [1234, 5678]},
 {'id': 'JEM061',
  'name': 'Quantitative Finance II.',
  'ects': '6',
  'teachers': [1234, 5678]}]

In [44]:
pd.DataFrame(l)

,id,name,ects,teachers
0,JEM005,Advanced Econometrics,6,"[3421, 1234]"
1,JEM207,Data Processing in Python,5,[3421]
2,JEM116,Applied Econometrics I.,6,[1234]
3,JEM059,Quantitative Finance I.,6,"[1234, 5678]"
4,JEM061,Quantitative Finance II.,6,"[1234, 5678]"


# HTML
standard web-page consists of:

* Browser-executed code (`front-end`)
    * HTML "DOM" structure - the website content
        * List of elements that are on website
        * Links to CSS classes, ids and
    * CSS stylesheets - website graphics
    * JavaScripts - website interactivity    

* Server-executed (`back-end`)
    * Server, database, app logic etc.
    * Not available for scraping!
    * May be available as API


## Web-scraping
* client side only
* Navigating HTML DOM by taking advantage of CSS structure

## DOM (Document Object Module):

In [46]:
html = '''
<html>
    <head>
        <title>Sample page</title>
    <script>
        function click_button() {
            alert('Hi guys, still enjoying Python?!')
        }
    </script>
    <style>
        #content div {
            color:black;
        }
        .firstRow {
            background-color:#ddd;
        }

        .normalRow {
            background-color:white;
        }
    </style>
    </head>
    
    <body>
        <div id="header">
            My page header
        </div>
        <div id="table_container">
            <table>
                <tr class="firstRow">
                    <td>name</td>
                    <td>number</td>
                </tr>
                <tr class="normalRow">
                    <td>B</td>
                    <td>2</td>
                </tr>
                <tr class="normalRow">
                    <td>C</td>
                    <td>3</td>
                </tr>
            </table>
        </div>
        <div id="button_container">
            <button id="btn" onclick="click_button()">Click Me!</button>
        </div
    </body>
</html>
'''
display(HTML(html))

NameError: name 'HTML' is not defined

In [49]:
soup = BeautifulSoup(html,'html')
soup

<html>
<head>
<title>Sample page</title>
<script>
        function click_button() {
            alert('Hi guys, still enjoying Python?!')
        }
    </script>
<style>
        #content div {
            color:black;
        }
        .firstRow {
            background-color:#ddd;
        }

        .normalRow {
            background-color:white;
        }
    </style>
</head>
<body>
<div id="header">
            My page header
        </div>
<div id="table_container">
<table>
<tr class="firstRow">
<td>name</td>
<td>number</td>
</tr>
<tr class="normalRow">
<td>B</td>
<td>2</td>
</tr>
<tr class="normalRow">
<td>C</td>
<td>3</td>
</tr>
</table>
</div>
<div id="button_container">
<button id="btn" onclick="click_button()">Click Me!</button>
</div>
</body></html>

In [50]:
rows = soup.findAll('tr',{'class','normalRow'})
rows

[<tr class="normalRow">
 <td>B</td>
 <td>2</td>
 </tr>,
 <tr class="normalRow">
 <td>C</td>
 <td>3</td>
 </tr>]

In [51]:
d = {}

for row in rows:
    key = row.findAll('td')[0].text
    val = int(row.findAll('td')[1].text)
    d[key] = val
pd.Series(d)

B    2
C    3
dtype: int64

In [52]:
d

{'B': 2, 'C': 3}

In [53]:
pd.Series({
    row.findAll('td')[0].text:int(row.findAll('td')[1].text) 
    for row in BeautifulSoup(html).findAll('tr',{'class':'normalRow'})})

B    2
C    3
dtype: int64

In [55]:
soup = BeautifulSoup(html)

In [56]:
row = soup.findAll('tr',{'class':'normalRow'})[0]

In [57]:
row

<tr class="normalRow">
<td>B</td>
<td>2</td>
</tr>

In [61]:
row.findAll('td')[0].text

'B'

In [60]:
int(row.findAll('td')[1].text)

2

In [62]:
{row.findAll('td')[0].text:int(row.findAll('td')[1].text) for row in soup.findAll('tr',{'class':'normalRow'})}

{'B': 2, 'C': 3}

## HTML Inspection
http://ies.fsv.cuni.cz/cs/node/51

In [63]:
import requests

# requests and internet communication

* `Client` asks/requests questions (your Jupyter client)
* `Server` replies/serve answers (your Jupyter server)


API = *Application Programming Interface*

very general term! Not only used in web communication

## HTTP requests

A most standard webserver communication channel around

A standard HTTP request contains:

* URL 

    * domain
    * route
    * parameters

* Request Type - GET, POST, PUT, DELETE (see below)

* Content specification - 
    * Application/JSON
    * Application/XML
    * text/html
    * text/css

* Content

* Outcoming data (will see below)

* Cookies 

* Status Code:

    * 200 - success
    * 404 - resource does not exist
    * 500 - the server failed during processing your request


1) REST API - use HTTP request and returns JSON

2) SOAP API - use HTTP request and returns XML

3) Website - use HTTP request and returns set of HTML, JavaScript, CSS and other files

### When to use?
* whenever more applications need to communicate
* user-friendly interface for complicated tasks - DEEP AI, Google Maps
* Data - Golemio, OpenStreetMaps

### GET request
* fast
* public
* data flow only one direction
* parameters via request adress

> https://www.google.com/search?q=how+to+understand+url+parameters&rlz=1C1GCEU_csCZ860CZ860&oq=how+to+understand+url+parameters&aqs=chrome..69i57j33i22i29i30l7.5237j0j4&sourceid=chrome&ie=UTF-8


In [64]:
r = requests.get('https://cs.wikipedia.org/wiki/Institut_ekonomick%C3%BDch_studi%C3%AD_Fakulty_soci%C3%A1ln%C3%ADch_v%C4%9Bd_Univerzity_Karlovy')
#plain request - like browser
r.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-night-mode-clientpref-0 vector-toc-available" lang="cs" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Institut ekonomických studií Fakulty sociálních věd Univerzity Karlovy – Wikipedie</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinne

In [66]:
soup = BeautifulSoup(r.text,'html')
tags=soup.findAll('span', {'class':"wd"})

In [67]:
tags

[<span class="wd"><span lang="cs">Budova IES FSV UK v Praze v Opletalově ulici</span></span>,
 <span class="wd"><a href="/wiki/Opletalova" title="Opletalova">Opletalova</a>, <a href="/wiki/Praha" title="Praha">Praha</a>, <a href="/wiki/%C4%8Cesko" title="Česko">Česko</a></span>,
 <span class="wd"><span></span><span class="coordinates"><a class="external text" href="//geohack.toolforge.org/geohack.php?language=cs&amp;pagename=Institut+ekonomick%C3%BDch+studi%C3%AD+Fakulty+soci%C3%A1ln%C3%ADch+v%C4%9Bd+Univerzity+Karlovy&amp;params=50.082219444444_N_14.431111111111_E_type:landmark"><span style="white-space:pre">50°4′55,99″ s. š.</span>, <span style="white-space:pre">14°25′52″ v. d.</span></a></span></span>,
 <span class="wd"><span class="sisterproject sisterproject-commons"><span class="sisterproject_image"><span typeof="mw:File"><a href="/wiki/Wikimedia_Commons" title="Wikimedia Commons"><img alt="Logo Wikimedia Commons" class="mw-file-element" data-file-height="1376" data-file-width="1

### POST request
* slow
* private
* both sides can send data

## Static pages x Dynamic pages x JavaScript-rendered pages

### Static

* pages that do not get updated instantly
* all information necessary for rendering a website is available after entering the URL
* It may ask the database, but the output is stable.
* all parameters within the adress!
* Typical example:
    
### JavaScript rendered: 
* Defacto static, but you cannot take advantage of HTML/CSS structure

### Dynamic content
* webpage instantly communicates with the webserver and the database
* 
* solution -> Selenium!

### Is this website static or dynamic?

1. Facebook
2. Sreality.cz
3. IES website



## How to chose data source for project

You need to know in advance what data you will download:

1. full or satisfactory access to API
2. the web-page is parsable (prefer not too much javascript)
3. plan to generate all requests

# APIs Example
### Get wiki data using GET

In [ ]:
#if time, return to geodata

# Lets start with a basic request

In [68]:
api_url = 'https://krcgc3uqga.execute-api.eu-central-1.amazonaws.com'
#this api implements three routers
# GET /time
# GET /stocks
# POST /hashme

In [69]:
route = 'time'
# route /ruːt/
response = requests.get(f'{api_url}/{route}')
response.json()


{'time': '2024-03-26T17:23:05.691Z'}

In [70]:
# route = stocks

route = 'stocks'
# route /ruːt/
response = requests.get(f'{api_url}/{route}')
print(response.json())

{'AAPL': 123.123}


In [71]:
route = "hashme"
url = f"https://krcgc3uqga.execute-api.eu-central-1.amazonaws.com/{route}"

payload = json.dumps({
  "name": "Jan Sila"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)

print(response.json())


{'hash': '9989540023c7d128cc66a374d79574b6'}


In [73]:
response = requests.get('https://en.wikipedia.org/wiki/Charles_University')
soup = BeautifulSoup(response.text)
div = soup.find('div',{'id':'mw-content-text'}) #  #mw-content-text > div > p:nth-child(10)texts)
article = ' '.join([p.text for p in div.find_all('p')])
print(article)

Charles University (CU; Czech: Univerzita Karlova, UK; Latin: Universitas Carolina; German: Karls-Universität), also known as Charles University in Prague or historically as the University of Prague (Latin: Universitas Pragensis), is the oldest and largest university in the Czech Republic.[2] It is one of the oldest universities in Europe in continuous operation.[3] Today, the university consists of 17 faculties located in Prague, Hradec Králové, and Plzeň. Charles University is in the top three universities in Central and Eastern Europe.[4][5] The institution has been ranked 200–300 in the world as one of the best universities.[6][7]
 The establishment of a medieval university in Prague was inspired by Holy Roman Emperor Charles IV.[8] He asked a friend and ally, Pope Clement VI, to do so. On 26 January 1347 the pope issued the bull establishing a university in Prague, modeled on the University of Paris, with the full (4) number of faculties, that is including a theological faculty. O

# Bonus example:

## GeoJSON

* One standardized data format for transferring geodata
* Plenty of geodata out there
* see for example http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json


In [74]:

verbose_request = requests.get('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json')


In [77]:
print(verbose_request.status_code)
dir(verbose_request)


404


['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [76]:
verbose_request.json()



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [78]:
#get already json
d = requests.get('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json').json()

d['features'][0]['properties']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [79]:
import branca
import folium

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 5)

def style_function(feature):
    gridvalue = feature['properties']['GRIDVALUE']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': colorscale(gridvalue)
    }

m = folium.Map(location=[50.085,14.45],zoom_start=11)
folium.GeoJson('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json',style_function=style_function).add_to(m)
m

ModuleNotFoundError: No module named 'branca'